In [220]:
import pandas as pd
import os
import numpy as np
import re
from collections import Counter
import matplotlib.pyplot as plt 
import seaborn as sb
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests 


# GETTING THE DATAFRAMES I'M GOING TO WORK WITH #

1. Download the base dataframe

In [221]:
df = pd.read_csv("../data/Dataset Global Fashion Brands Brand Equity Ranking Growth Rate  COO ROO 2001-2021.csv")
df.sample(10)

,BrandName,BrandOriginCountry,BrandOriginRegion,BrandSector,BrandSubSector,Rank2001,Rank2002,Rank2003,Rank2004,Rank2005,...,GrowthRate2012,GrowthRate2013,GrowthRate2014,GrowthRate2015,GrowthRate2016,GrowthRate2017,GrowthRate2018,GrowthRate2019,GrowthRate2020,GrowthRate2021
3,L’Oréal,France,Europe,Fashion,Cosmetics,NaN,54.0,47.0,49.0,52.0,...,1.40,11.94,2.92,6.26,1.22,-2.34,4.01,4.39,8.32,-0.41
18,Cartier,Switzerland,Europe,Fashion,Luxury,NaN,NaN,NaN,91.0,89.0,...,14.93,25.51,8.00,6.38,-2.35,-2.47,1.31,7.14,-8.52,8.90
24,Levi's,U.S.A.,America,Fashion,Apparel,67.0,73.0,77.0,85.0,96.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Gucci,Italy,Europe,Fashion,Luxury,50.0,52.0,53.0,59.0,49.0,...,7.79,7.46,2.31,-14.47,5.66,6.22,29.82,23.23,-1.72,6.26
10,Puma,Germany,Europe,Fashion,Sportswear,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Boss,Germany,Europe,Fashion,Apparel,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.07,-100.00,NaN,NaN,NaN,NaN,NaN
2,Hermès,France,Europe,Fashion,Luxury,NaN,NaN,73.0,79.0,82.0,...,15.23,23.20,17.87,21.91,17.26,10.73,15.21,9.46,0.23,20.26
28,Avon,U.S.A.,America,Fashion,Cosmetics,60.0,62.0,57.0,58.0,59.0,...,-4.19,-10.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Sephora,France,Europe,Fashion,Cosmetics,100.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,Polo,U.S.A.,America,Fashion,Luxury,85.0,95.0,95.0,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


2. Web scrapping for 2nd dataframe

''' The next comments you'll find are unsuccessful tries to webscraping an url. It didn't work cause they're calling themselfs, tried to do it for a long time and Fer found a way 
to download the csv directly so all the next steps until reading the next csv didn't serve but I didn't want to 
delete it cause I spent 3 hours trying to scrap the website.'''

In [222]:
'''url = 'https://www.brandwatch.com/brandwatch-index/luxury-fashion'
res = requests.get(url, timeout=5)
html = res.content
soup = BeautifulSoup(html, 'html.parser')'''

"url = 'https://www.brandwatch.com/brandwatch-index/luxury-fashion'\nres = requests.get(url, timeout=5)\nhtml = res.content\nsoup = BeautifulSoup(html, 'html.parser')"

In [223]:
'''def get_columns(soup):
    index = soup.find("div", {"class": "index-table__row__pos"})
    ind = index.find_all("h5")
    ind_processed = [i.getText() for i in ind]
    return ind_processed'''
    

'def get_columns(soup):\n    index = soup.find("div", {"class": "index-table__row__pos"})\n    ind = index.find_all("h5")\n    ind_processed = [i.getText() for i in ind]\n    return ind_processed'

In [224]:
'''get_columns(soup)'''

'get_columns(soup)'

In [225]:
'''ds = soup.find_all("li", {"class": "lindex-table__row index-table__row--body js-c-si-row"})
ds'''

'ds = soup.find_all("li", {"class": "lindex-table__row index-table__row--body js-c-si-row"})\nds'

In [226]:
'''url = "https://www.brandwatch.com/wp-content/uploads/2020/01/LuxFashion-Q42019.csv?rev=1698422964802"
res = requests.get(url)
res'''

'url = "https://www.brandwatch.com/wp-content/uploads/2020/01/LuxFashion-Q42019.csv?rev=1698422964802"\nres = requests.get(url)\nres'

In [227]:
'''res.content'''

'res.content'

In [228]:
'''import pandas as pd
url = 'https://www.brandwatch.com/brandwatch-index/luxury-fashion'
res = requests.get(url)
soup = BeautifulSoup(res.content, "html.parser")
soup'''


'import pandas as pd\nurl = \'https://www.brandwatch.com/brandwatch-index/luxury-fashion\'\nres = requests.get(url)\nsoup = BeautifulSoup(res.content, "html.parser")\nsoup'

In [229]:
'''ds = soup.find_all("li")
ds[20]'''

'ds = soup.find_all("li")\nds[20]'

2. 1 Finding another way to get the needed data.

    In this case downloaded as csv and imported with pandas

In [230]:
df_2= pd.read_csv("../data/LuxFashion-Q42019.csv")
df_2.sample(5)

,Brand,Social Visibility,Web Visibility,Sentiment,Growth,Search,Total
26,Dolce & Gabbana,6.97,7.76,40.87,0.69,1.99,58.28
25,Salvatore Ferragamo,1.62,8.06,54.06,9.64,5.13,78.51
24,Christian Louboutin,4.58,12.43,50.67,0.00,11.71,79.39
23,DKNY,1.38,4.49,67.95,0.00,6.38,80.20
12,Cartier,10.44,34.93,41.27,19.37,16.83,122.84


3. Get the 3rd Dataframe by webscrapping

In [231]:
url = 'https://fashionunited.com/i/top200'
res = requests.get(url, timeout=5)
html = res.content
soup = BeautifulSoup(html, 'html.parser')

In [232]:
''' Function to get html content from a web with bs4. '''

def download_html(url):
    res = requests.get(url, timeout=5)
    html = res.content
    return BeautifulSoup(html, 'html.parser')

In [233]:
url = 'https://fashionunited.com/i/top200'
soup = download_html(url)

In [234]:
'''Function to get a table and transform it into a dataframe from an already requested link.'''

def get_table_web(soup):
    table =  soup.find("table")
    rows = table.find_all('tr')
    data = []
    for row in rows:
        cols = row.find_all('td')
        cols = [col.text for col in cols]
        data.append(cols)
    ds = pd.DataFrame(data, columns=['Index','Rank', 'Company', 'Mcap $', 'Type'])
    return ds.drop(['Index'], axis=1)

In [235]:
'''Calling of the function'''

df_3 = get_table_web(soup)
df_3.sample(30)

,Rank,Company,Mcap $,Type
32,32,Patek Phillippe,$7.62 b,Private
111,111,Russell Athletic,$1.06 b,Private
150,150,St. John,$986 m,Private
101,101,The Buckle Inc.,$1.68 b,Public
120,120,Selfridges Group,$2.09 b,Private
156,156,Marc O Polo,$876 m,Private
129,129,Monsoon Accessorize,$1.69 b,Private
160,160,G-III Apparel Group,$743 m,Public
118,118,Bosco di Ciliegi,$2.13 b,Private
4,4,Christian Dior,$108.7 b,Public


4. Export the last dataframe to csv.

In [236]:
df_3.to_csv(f"../data/100_fashion_money.csv")

5. Get a list of participating brand from the 3 more important fashion weeks: NYFW, Milan FW, Paris FW

         Milan Fashion week info extraction. Participating brands.

In [237]:
url_2 = 'https://www.wallpaper.com/fashion-beauty/milan-fashion-week-aw-2023-highlights'

In [238]:
''' Reusing the by calling it'''

soup = download_html(url_2)

In [239]:
ds =  soup.find("div", {"id": "article-body"})
titles = ds.find_all("h2")
brands = [i.getText() for i in titles]
FWM = [i for i in brands if "Week" not in i]
FWM


['Giorgio Armani',
 'Bottega Veneta',
 'Dolce & Gabbana',
 'MSGM',
 'Ferragamo',
 'Ferrari',
 'Jil Sander',
 'Andreadamo',
 'Gucci',
 'Sportmax',
 'Tod’s',
 'Blumarine',
 'Emporio Armani',
 'Prada',
 'Max Mara',
 'Etro',
 'No. 21',
 'Fendi',
 'Diesel']

    '''Now, it's time to encapsulate the code. I've been doing it and deleting the process to get there, but maybe it's a good idea to leave it. '''

In [240]:
''' Function to get titles from an article body'''

def titles_article(soup):
    ds =  soup.find("div", {"id": "article-body"})
    titles = ds.find_all("h2")
    brands = [i.getText() for i in titles]
    return [i for i in brands if "Week" not in i]

In [241]:
MFW = titles_article(soup)
MFW

['Giorgio Armani',
 'Bottega Veneta',
 'Dolce & Gabbana',
 'MSGM',
 'Ferragamo',
 'Ferrari',
 'Jil Sander',
 'Andreadamo',
 'Gucci',
 'Sportmax',
 'Tod’s',
 'Blumarine',
 'Emporio Armani',
 'Prada',
 'Max Mara',
 'Etro',
 'No. 21',
 'Fendi',
 'Diesel']

         Paris Fashion week info extraction. Participating brands.

In [242]:
''' Reusing the function by calling it'''

url_3 = "https://www.fhcm.paris/en/paris-fashion-week/calendar"
soup = download_html(url_3)

In [243]:
ds =  soup.find_all("div", {"class": "cal-item shown"})
a =[i.find("a") for i in ds]
names = [i.find("h3") for i in a]
brands_filtered = [i.getText().title() for i in names]
brands_filtered

['Marie Adam-Leenaerdt',
 'Weinsanto',
 'Benjamin Benmoyal',
 'Vaquera',
 'Mossi',
 'Pierre Cardin',
 'Barbara Bui',
 'Peter Do',
 'Niccolò Pasqualetti',
 'Cfcl',
 'Victoria/Tomas',
 'Mame Kurogouchi',
 'Christian Dior',
 'Dawei',
 'Florentina Leitner',
 'Anrealage',
 'Pressiat',
 'Germanier',
 'Heliot Emil',
 'Saint Laurent',
 'Courreges',
 'Reverie By Caroline Hù',
 'The Row',
 'Marni',
 'Minuit',
 'Dries Van Noten',
 'Kimhekim',
 'Cecilie Bahnsen',
 'Litkovska',
 'Undercover',
 'Didu',
 'Acne Studios',
 'Balmain',
 'Uma Wang',
 'Veronique Leroy',
 'Lanvin',
 'Rabanne',
 'Gauchere',
 'Chloé',
 'Givenchy',
 'Nehera',
 'Christopher Esber',
 'Rick Owens',
 'Schiaparelli',
 'Isabel Marant',
 'Alexis Mabille',
 'Alexandre Vauthier',
 'Leonard Paris',
 'Loewe',
 'Maitrepierre',
 'Issey Miyake',
 'Jitrois',
 'Giambattista Valli',
 'Lutz Huelle',
 'Nina Ricci',
 'Enfants Riches Deprimes',
 'Victoria Beckham',
 'Situationist',
 'Yohji Yamamoto',
 'Coperni',
 'Junya Watanabe',
 'Carven',
 'Rui

    ''' Now let's encapsulate'''

In [244]:
''' Get brands from containers in a calendar in html. '''

def brands_filtered(soup):
    ds =  soup.find_all("div", {"class": "cal-item shown"})
    a =[i.find("a") for i in ds]
    names = [i.find("h3") for i in a]
    return [i.getText().title() for i in names]

In [245]:
''' Calling the functions'''

PFW = brands_filtered(soup)
PFW

['Marie Adam-Leenaerdt',
 'Weinsanto',
 'Benjamin Benmoyal',
 'Vaquera',
 'Mossi',
 'Pierre Cardin',
 'Barbara Bui',
 'Peter Do',
 'Niccolò Pasqualetti',
 'Cfcl',
 'Victoria/Tomas',
 'Mame Kurogouchi',
 'Christian Dior',
 'Dawei',
 'Florentina Leitner',
 'Anrealage',
 'Pressiat',
 'Germanier',
 'Heliot Emil',
 'Saint Laurent',
 'Courreges',
 'Reverie By Caroline Hù',
 'The Row',
 'Marni',
 'Minuit',
 'Dries Van Noten',
 'Kimhekim',
 'Cecilie Bahnsen',
 'Litkovska',
 'Undercover',
 'Didu',
 'Acne Studios',
 'Balmain',
 'Uma Wang',
 'Veronique Leroy',
 'Lanvin',
 'Rabanne',
 'Gauchere',
 'Chloé',
 'Givenchy',
 'Nehera',
 'Christopher Esber',
 'Rick Owens',
 'Schiaparelli',
 'Isabel Marant',
 'Alexis Mabille',
 'Alexandre Vauthier',
 'Leonard Paris',
 'Loewe',
 'Maitrepierre',
 'Issey Miyake',
 'Jitrois',
 'Giambattista Valli',
 'Lutz Huelle',
 'Nina Ricci',
 'Enfants Riches Deprimes',
 'Victoria Beckham',
 'Situationist',
 'Yohji Yamamoto',
 'Coperni',
 'Junya Watanabe',
 'Carven',
 'Rui

         New York week info extraction. Participating brands.

In [246]:
''' Reusing the function by calling it'''

url_4 = "https://www.vogue.com/fashion-shows"
soup = download_html(url_4)

In [247]:
''' I tried different ways to get the information I needed because at first it gave me a hole lot more
than I needed. This is the final way that worked. '''

#ds =  soup.find_all("nav", {"class": "NavigationWrapper-bFftAs fKSZRK GroupedNavigationGroup-KHsBC cKGNRO grouped-navigation__group"})
li = soup.find_all("li", {"class":"NavigationListItemWrapper-cxLZKD iAYgTw link--secondary navigation__list-item"})
names = [i.getText() for i in li]
names
#a = [i.find("a", {"class":"NavigationInternalLink-cWEaeo knabMb grouped-navigation__link link--secondary navigation__link"}) for i in li]
#a


['A. Potts',
 'A.L.C.',
 'A.P.C.',
 'A.W.A.K.E. Mode',
 'Acne Studios',
 'Adam Lippes',
 'Adeam',
 'AG',
 'AGL',
 'Ahluwalia',
 'Akris',
 'Alaïa',
 'Alberta Ferretti',
 'Alejandra Alonso Rojas',
 'Alessandra Rich',
 'Alexander McQueen',
 'Alexandre Vauthier',
 'Alexis Mabille',
 'Alice + Olivia',
 'All-In',
 'Altuzarra',
 'Ambush',
 'Andreadamo',
 'Andreas Kronthaler for Vivienne Westwood',
 'Ann Demeulemeester',
 'Anna October',
 'Anna Sui',
 'Anrealage',
 'Anteprima',
 'Antonio Marras',
 'Apiece Apart',
 'Armarium',
 'Arthur Arbesser',
 'ASAI',
 'Ashish',
 'Ashley Williams',
 'Ashlyn',
 'Atlein',
 'ATXV',
 'Avavav',
 'Axel Arigato',
 'AZ Factory',
 'Bach Mai',
 'Badgley Mischka',
 'Balenciaga',
 'Bally',
 'Balmain',
 'Barbara Tfank',
 'Batsheva',
 'Bevza',
 'Bibhu Mohapatra',
 'Bite Studios',
 'Blumarine',
 'Bora Aksu',
 'Bottega Veneta',
 'Brandon Maxwell',
 'Bronx and Banco',
 'Brunello Cucinelli',
 'Burberry',
 'By Malene Birger',
 'Câllas Milano',
 'Calvin Luo',
 'Carolina Herrer

    ''' Now let's encapsulate. '''

In [248]:
''' Function to get the brand names from a nav bar'''

def get_nav_brands(soup):
    li = soup.find_all("li", {"class":"NavigationListItemWrapper-cxLZKD iAYgTw link--secondary navigation__list-item"})
    return [i.getText() for i in li]

In [249]:
''' Calling the function and assigning the name of New York Fashion Week. '''

NYFW = get_nav_brands(soup)
NYFW

['A. Potts',
 'A.L.C.',
 'A.P.C.',
 'A.W.A.K.E. Mode',
 'Acne Studios',
 'Adam Lippes',
 'Adeam',
 'AG',
 'AGL',
 'Ahluwalia',
 'Akris',
 'Alaïa',
 'Alberta Ferretti',
 'Alejandra Alonso Rojas',
 'Alessandra Rich',
 'Alexander McQueen',
 'Alexandre Vauthier',
 'Alexis Mabille',
 'Alice + Olivia',
 'All-In',
 'Altuzarra',
 'Ambush',
 'Andreadamo',
 'Andreas Kronthaler for Vivienne Westwood',
 'Ann Demeulemeester',
 'Anna October',
 'Anna Sui',
 'Anrealage',
 'Anteprima',
 'Antonio Marras',
 'Apiece Apart',
 'Armarium',
 'Arthur Arbesser',
 'ASAI',
 'Ashish',
 'Ashley Williams',
 'Ashlyn',
 'Atlein',
 'ATXV',
 'Avavav',
 'Axel Arigato',
 'AZ Factory',
 'Bach Mai',
 'Badgley Mischka',
 'Balenciaga',
 'Bally',
 'Balmain',
 'Barbara Tfank',
 'Batsheva',
 'Bevza',
 'Bibhu Mohapatra',
 'Bite Studios',
 'Blumarine',
 'Bora Aksu',
 'Bottega Veneta',
 'Brandon Maxwell',
 'Bronx and Banco',
 'Brunello Cucinelli',
 'Burberry',
 'By Malene Birger',
 'Câllas Milano',
 'Calvin Luo',
 'Carolina Herrer

''' The plan with the lists of participating brands on the 3 Fashion Weeks is to create 3 columns with the fashion weeks names and compare the lists with the brands in the base dataframe and assign to these new columns a boolean value if the brand participated in that specific fashion Week. '''

6. Get a list of Designers of the brands from the base dataset. And if possible the year of "icorporación".

In [250]:
''' Reusing the function by calling it'''

url_5 = "https://highxtar.com/la-cfda-anuncia-los-nominados-para-el-2020/"
soup = download_html(url_5)

In [251]:
''' Function to get list of Designers. This one was harsher as every needed item was a "br" inside a "p"
so it wasn't possible to extract specifically the item. But founnf a workaround by indexing.'''

def get_designers(soup):
    ds = soup.find("div", {"class":"content__main__content"})
    p = ds.find_all("p")
    text = [i.getText() for i in p]
    lists = [i.split("\n") for i in text if "CFDA" not in i]
    return [i[1:] for i in lists]

In [252]:
designers = get_designers(soup)
designers

[['', '', '', ''],
 ['Mary-Kate and Ashley Olsen, The Row',
  'Brandon Maxwell',
  'Gabriela Hearst',
  'Marc Jacobs',
  'Tom Ford'],
 ['Emily Adams Bode, Bode',
  'Kerby Jean-Raymond, Pyer Moss',
  'Todd Snyder',
  'Thom Browne',
  'Tom Ford'],
 ['Mary-Kate and Ashley Olsen, The Row',
  'Gabriela Hearst',
  'Stuart Vevers, Coach',
  'Jennifer Fisher, Jennifer Fisher Jewelry',
  'Telfar Clemens, Telfar'],
 ['Christopher John Rogers',
  'Kenneth Nicholson',
  'Peter Do',
  'Reese Cooper',
  'Sarah Staudinger and George Augusto, STAUD'],
 ['Daniel Lee, Bottega Veneta',
  'Dries Van Noten',
  'Miuccia Prada, Prada',
  'Pierpaolo Piccioli, Valentino',
  'Rick Owens'],
 ['Craig Green',
  'Dries Van Noten',
  'Jonathan Anderson, Loewe',
  'Kim Jones, Dior',
  'Virgil Abloh, Louis Vuitton']]